## Speed test

In this notebook we compare the execution speed of the different pricing model and the imporvement made to speed them up.

In [1]:
import numpy as np
import pandas as pd
import ast
import os
import sys
from os.path import dirname as up

# Important directories
code_dir = os.path.dirname(os.getcwd())
deep_cal_dir = os.path.dirname(os.path.dirname(os.getcwd()))

# Allows to import my own module
sys.path.insert(0, code_dir)

from pricing.liftedheston import Pricer_Lifted_Heston
from pricing.legacyliftedheston import Legacy_Pricer_Lifted_Heston
from pricing.linear_iv import linear
from pricing.newton_iv import newton_raphson
from pricing.brent_iv import brent
from pricing.heston_fourier import price_heston_fourier
from pricing.legacy import price_heston_mc_legacy
from pricing.heston_mc import price_heston_mc

### Linear implied volatility

In [2]:
%timeit linear(S_=0.8,K_=1.,r_=0.03,tau_=0.8,price_=3)

11.7 µs ± 309 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Newton implied volatility 

In [3]:
%timeit newton_raphson(S_=0.8,K_=1.,r_=0.03,tau_=0.8, sigma0_= 0.1,price_=3, epsilon_=1e-5)

657 µs ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Brent implied Volatility 

In [4]:
%timeit brent(S_=0.8,K_=1.,r_=0.03,tau_=0.8, sigma0_= 0.1,sigma1_=0.5,price_=3, epsilon_=1e-5)

1.07 ms ± 30.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Heston Fourier

In [5]:
%timeit price_heston_fourier(K_=1.,alpha_=1.,r_=0.05,tau_=0.8,kappa_=1.5,S0_=0.8,theta_=0.1,rho_=-0.05,sigma_=0.3,V0_=0.1,L_=50)

4.01 ms ± 73.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Heston Monte Carlo

Legacy version

In [6]:
%timeit price_heston_mc_legacy(kappa_=1.5,theta_=0.1,sigma_=0.3,rho_=-0.05,r_=0.05,T_=0.8,L_=100,V0_=0.1,S0_=0.8,K0_=1,N_=1000)

831 ms ± 36.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Speed up mc

In [7]:
%timeit price_heston_mc(kappa_=1.5,theta_=0.1,sigma_=0.3,rho_=-0.05,r_=0.05,T_=0.8,L_=100,V0_=0.1,S0_=0.8,K0_=1,N_=1000)

45.8 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit Pricer_Lifted_Heston(1,0.03,0.8,0.8,rho=-0.7,lamb=0.3,theta=0.02,nu=0.3,V0=0.02,N=20,rN=2.5,alpha=0.1+1/2,M=200,L_=50)

221 ms ± 7.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit Legacy_Pricer_Lifted_Heston(1,0.03,0.8,0.8,rho=-0.7,lamb=0.3,theta=0.02,nu=0.3,V0=0.02,N=20,rN=2.5,alpha=0.1+1/2,M=200,L_=50)

852 ms ± 8.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
